# Step 0: Import libraries

In [1]:
import os
from openai import OpenAI 
from dotenv import load_dotenv
from IPython.display import display, Markdown
from tavily import TavilyClient
from pydantic import BaseModel
from typing import List, Optional, Literal

In [3]:
# import keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
if not openai_api_key:
   raise ValueError("OPENAI_API_KEY not found in .env file")
if not tavily_api_key:
   raise ValueError("TAVILY_API_KEY not found in .env file")

# Step 2: Setup output format using Pydantic's BaseModel

In [4]:
class ResearchValidation(BaseModel):
    is_valid: bool
    reason: str | None = None

#class1: define class SubTopicPlan: This will return details of each subtopic, including 1) id 2) title 3) guiding question (list of string) 4) depth (overview/detailed/technical) 5) description 6) notes (optional)
class SubTopicPlan(BaseModel): #from Topic Splitter to Researcher Agent
    id: str
    title: str
    guiding_question: List[str] = []
    depth: Literal['overview', 'detailed', 'technical']
    description: str
    notes: Optional[str]

# Class2: define class TopicSplitResult: This will returns: 1) Original topic 2) list of subtopic, which its details will then be defined in another class

class TopicSplitterResult(BaseModel): #output from Topic Splitter
    original_topic: str
    subtopics: List[SubTopicPlan]



# class3: EvidenceItem: This will return details of each evidence item, including 1) id 2) title 3) type (article/book/video) 4) url 5) summary 6) relevance_score (0-100)
class EvidenceItem(BaseModel):
    source: str
    quote_or_paraphrase: str
    relevance_note: Optional[str]   = None

# class 4: ResearcherOutput: This will return details of research result for each subtopic, including 1) subtopic_id 2) list of evidence items (defined in class EvidenceItem)
class ResearcherOutput(BaseModel):
    subtopic_id: str
    subtopic_title: str
    summary: str
    analysis: str
    evidence_items: List[EvidenceItem] = []

# class 5: SynthesizerResult: This will return details of synthesized research for each subtopic, including 1) subtopic_id 2) all SubtopicPlans 3) all ResearcherOutputs
class SynthesizerResult(BaseModel):
    user_topic: str
    subtopic_plans: List[SubTopicPlan]
    researcher_outputs: List[ResearcherOutput]

# class 6: FinalReportSection and FinalReport: This will return details of final report, including 1) section heading 2) section content 3) list of subtopic_ids integrated in this section

class FinalReportSection(BaseModel):
    """A single section in the final report shown to the user."""
    heading: str                 # section title
    content: str                 # main text
    subtopic_ids: List[str] = [] # which subtopics this section integrates

# class 7: FinalReport: This will return details of final report, including 1) overall topic/question 2) executive summary 3) list of FinalReportSections 4) list of key points (bullet-like main takeaways) 5) list of deduped sources (EvidenceItem) 6) list of known limitations/caveats
class FinalReport(BaseModel):
    """Final artifact that you send to the UI (Gradio)."""
    schema_version: str = "1.0"
    topic: str                   # overall topic / question
    summary: str                 # executive summary
    sections: List[FinalReportSection]
    key_points: List[str]        # bullet-like main takeaways
    # EvidenceItem is the same one you already defined for ResearcherOutput
    sources: List[EvidenceItem] = []  # deduped list of sources
    limitations: List[str] = []         # known limitations / caveats


# Step 3: Instantiate agents


In [5]:
from agents import Agent, Runner, WebSearchTool, InputGuardrail, GuardrailFunctionOutput

In [6]:
# Create Input Guardrail Agent
input_guardrail_agent = Agent(
    name="InputGuardrail",
    instructions="You are an input validation agent. Ensure that the user input is not asking anything inapproropriate.",
    output_type=ResearchValidation
)

async def input_guardrail(ctx, agent, input_data):
    result = await Runner.run(input_guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(ResearchValidation)
    return GuardrailFunctionOutput(
        output_info = final_output,
        tripwire_triggered = not final_output.is_valid
    )

In [7]:
# Create topic splitter agent
topic_splitter_agent = Agent(
    name="TopicSplitter",
    instructions="You are a topic splitting agent. Your task is to break down a complex topic into manageable subtopics for research.",
    output_type=TopicSplitterResult,
    input_guardrails=[InputGuardrail(guardrail_function=input_guardrail)],
)

In [8]:
# create researcher agent
researcher_agent = Agent(
    name = "ResearcherAgent",
    instructions = "You are a researcher agent. Your task is use WebSearchTool to find and summarize relevant research papers.",
    output_type = ResearcherOutput,
    tools= [WebSearchTool()]
)

In [9]:
# Create synthesizer agent
synthesizer_agent = Agent(
    name="SynthesizerAgent",
    instructions="You are a synthesizer agent. Your task is to synthesize research findings from multiple subtopics into a coherent report.",
    output_type=SynthesizerResult,
    handoff_description="Synthesize the research findings into a coherent report."
)

In [10]:
# Optimizer Agent
optimizer_agent = Agent(
    name="OptimizerAgent",
    instructions="You are an optimizer agent. Your task is to optimize and structure the final report based on synthesized research findings.",
    output_type=FinalReport,
    handoff_description="Optimize and structure the final report for clarity and coherence."
)

In [19]:
# define handoff agent
triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a router. Choose exactly one of the handoff agents.\n"
        "- If the user explicitly asks to break a topic into subtopics, use TopicSplitter.\n"
        "- If the user asks for an explanation, detailed answer, or full report, "
        "route to OptimizerAgent (which produces a FinalReport).\n"
    ),
    handoffs=[topic_splitter_agent, researcher_agent, synthesizer_agent, optimizer_agent],
    output_type=FinalReport,
)


# Step 4: Run the agent orchestration

In [23]:
from agents import Runner

async def main():
    user_input = "Explain the implications of quantum computing on modern cryptography."
    
    print("🔄 Running agent orchestration...")
    result = await Runner.run(triage_agent, user_input)
    final_report = result.final_output_as(FinalReport)
    
    # Display the complete final report
    display(Markdown(f"# 📊 Final Report: {final_report.topic}"))
    display(Markdown("---"))
    
    # Executive Summary
    display(Markdown(f"## 📝 Executive Summary"))
    display(Markdown(final_report.summary))
    display(Markdown("---"))
    
    # Key Points
    display(Markdown(f"## 🔑 Key Takeaways"))
    for i, point in enumerate(final_report.key_points, 1):
        display(Markdown(f"{i}. {point}"))
    display(Markdown("---"))
    
    # Sections
    display(Markdown(f"## 📚 Detailed Analysis"))
    for section in final_report.sections:
        display(Markdown(f"### {section.heading}"))
        display(Markdown(section.content))
        if section.subtopic_ids:
            display(Markdown(f"*Related subtopics: {', '.join(section.subtopic_ids)}*"))
        display(Markdown(""))
    display(Markdown("---"))
    
    # Sources
    if final_report.sources:
        display(Markdown(f"## 🔗 Sources"))
        for i, source in enumerate(final_report.sources, 1):
            display(Markdown(f"{i}. **{source.source}**"))
            display(Markdown(f"   - {source.quote_or_paraphrase}"))
            if source.relevance_note:
                display(Markdown(f"   - *Note: {source.relevance_note}*"))
        display(Markdown("---"))
    
    # Limitations
    if final_report.limitations:
        display(Markdown(f"## ⚠️ Limitations & Caveats"))
        for limitation in final_report.limitations:
            display(Markdown(f"- {limitation}"))
    
    print("\n✅ Report generation complete!")
    return final_report

In [24]:
await main()

🔄 Running agent orchestration...


# 📊 Final Report: Implications of Quantum Computing on Modern Cryptography

---

## 📝 Executive Summary

Quantum computing poses a significant challenge to modern cryptography, particularly public key systems such as RSA and ECC, by potentially enabling the rapid solution of mathematical problems that underlie their security. Symmetric cryptography is less vulnerable but still requires larger key sizes to maintain security. The growing capabilities of quantum computers are driving urgent research into quantum-resistant 'post-quantum' cryptographic algorithms.

...

- Quantum computers capable of breaking current cryptography are not available yet; timelines for their development are uncertain.

- Current post-quantum cryptographic algorithms may face unforeseen vulnerabilities or implementation challenges.

- Transitioning global infrastructure to new cryptosystems is complex and may introduce additional risks and interoperability issues.


✅ Report generation complete!


FinalReport(schema_version='v1', topic='Implications of Quantum Computing on Modern Cryptography', summary="Quantum computing poses a significant challenge to modern cryptography, particularly public key systems such as RSA and ECC, by potentially enabling the rapid solution of mathematical problems that underlie their security. Symmetric cryptography is less vulnerable but still requires larger key sizes to maintain security. The growing capabilities of quantum computers are driving urgent research into quantum-resistant 'post-quantum' cryptographic algorithms.", sections=[FinalReportSection(heading='The Quantum Threat to Classical Cryptographic Algorithms', content='Most widely used cryptographic algorithms today, such as RSA, Diffie-Hellman, and Elliptic Curve Cryptography (ECC), rely on the computational difficulty of problems like integer factorization and discrete logarithms. Quantum algorithms—particularly Shor’s algorithm—could efficiently solve these problems, enabling an adve